In [2]:
import math
import numpy as np
import os
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.utils.data as data_utils

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

from beerai.config import DATA_DIR

In [3]:
def load_vectors():
    vec_file = os.path.join(DATA_DIR, "processed/recipe_vecs.h5")
    with pd.HDFStore(vec_file, "r") as store:
        vectors = store.get("/vecs")
    return vectors

def load_vocab():
    vocab_file = os.path.join(DATA_DIR, "processed/vocab.pickle")
    with open(vocab_file, "rb") as f:
        vocab = pickle.load(f)
    inv_vocab = {v: k for k,v in vocab.items()}
    return vocab, inv_vocab

In [4]:
recipes = load_vectors()

In [5]:
recipes.head()

name,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,678.0,679.0,680.0,681.0,682.0,683.0,684.0,685.0,686.0,boil_time
recipe_id,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,60.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,60.0
5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.274902,0.0,0.0,0.0,0.0,90.0
7,1.553145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,60.0
8,1.240266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,60.0


In [6]:
recipes["boil_time"] = recipes.boil_time.clip(upper=120)

In [7]:
scaler = MinMaxScaler()
recipes_scaled = scaler.fit_transform(recipes)

Notes on sparse autoencoders

- [Discussion on quora](https://www.quora.com/When-training-an-autoencoder-on-very-sparse-data-how-do-you-force-the-decoder-to-reconstruct-mostly-zeros-rather-than-always-just-reconstructing-the-average)
  - > So, if you're including a bias term in your autoencoders, I recommend removing the bias and attempting training again.
  - > I encountered this problem recently. I found that, similar to what Eric described, it is a problem with SGD getting stuck in bad local optima. A couple things helped: (1) using conjugate gradients or AdaGrad, either of which will find the path to the true minimum without getting stuck as much as plain SGD will; (2) using a combined cross-entropy & mean-squared-error loss function (assuming that you can model your data as binary vectors or as probability distributions) pulls things -- ever so slightly -- in the right directions better than either alone would.
  - > I decided to progressively lower the learning rate and then I got good results. So that's my tip, lower your learning rate until you get better results. (For example, I'm using now: 0.0000005 as the initial rate.) And don’t forget to normalize your data!
  - > You shouldn't need to do anything special for this. Standard good practices for initialization and training should take care of it.
  - > We were able to reproduce the original image (not get the average) by using AdamOptimizer and lowering the learning rate.
  - [Notes](http://web.stanford.edu/class/cs294a/sae/sparseAutoencoderNotes.pdf) from Stanford CS294a (Andrew Ng)  

In [8]:
class AutoEncoder(nn.Module):
    def __init__(
        self,
        input_dim,
        compression_dim,
        factor_per_layer,
    ):

        super().__init__()
        self.compression_dim = compression_dim

        comp_layers, decomp_layers = self.gen_layers_by_factor(
            input_dim, compression_dim, factor_per_layer
        )

        self.encoder = nn.Sequential(*comp_layers)
        self.decoder = nn.Sequential(*decomp_layers)
 
    def gen_layers_by_factor(self, input_dim, compression_dim, factor_per_layer):
        
        cur_dim = input_dim
        compress_layers = []
        decompress_layers = []
        n_iters = math.ceil(math.log(input_dim / compression_dim, factor_per_layer))
        
        for i in range(n_iters):
            new_dim = max(cur_dim // factor_per_layer, compression_dim)
            compress_layers.extend([nn.Linear(cur_dim, new_dim, bias=False), nn.ReLU(True)])
            decompress_layers.extend([nn.ReLU(True), nn.Linear(new_dim, cur_dim, bias=False)])
            cur_dim = new_dim
        decompress_layers = decompress_layers[::-1]
        # Replace final layer with sigmoid/tanh. Should match the input scaling range 
        compress_layers[-1] = nn.Sigmoid()
        decompress_layers[-1] = nn.Sigmoid()

        return compress_layers, decompress_layers

    def forward(self, X):
        encoded = self.encoder(X)
        decoded = self.decoder(encoded)
        return decoded

    def encode(self, X):
        X = torch.tensor(X, dtype=torch.float)
        X = self.encoder(X)
        return pd.DataFrame(X.detach().numpy())
    
    def decode(self, X):
        X = torch.tensor(X, dtype=torch.float)
        X = self.decoder(X)
        return pd.DataFrame(X.detach().numpy())

In [9]:
def train(model, X, batch_size, num_epochs, learning_rate=1e-3, beta=1):

    if isinstance(X, pd.DataFrame):
        X = X.values

    mse = nn.MSELoss()
    kldiv = nn.KLDivLoss()
    optimizer = torch.optim.Adam(
        model.parameters(), lr=learning_rate, weight_decay=1e-5
    )

    #features = torch.tensor(X, dtype=torch.float)
    data = torch.tensor(X, dtype=torch.float)
    
    #train = data_utils.TensorDataset(features)
    
    # Shuffle used to ensure randomized selection
    #train_loader = data_utils.DataLoader(
    #    train, batch_size=batch_size, shuffle=True
    #)

    for epoch in range(num_epochs):
        losses = []
        #for i, batch in enumerate(train_loader):
            # ===================forward=====================
            #data = batch[0]
        decoded = model.forward(data)
        length = decoded.shape[0]

        if length < batch_size:
            continue
        mse_loss = mse(decoded, data)
        kld_loss = kldiv(decoded, data)
        loss = mse_loss + beta * kld_loss


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.data)

        print(f"epoch [{epoch + 1}/{num_epochs}], loss:{sum(losses)/len(losses):.6f}")

In [10]:
input_dims = len(recipes.columns)
# first guess
compress_dims = 50
# factor to reduce by each layer
factor_per_layer = 2
beer_ae = AutoEncoder(input_dims, compress_dims, factor_per_layer)

In [11]:
batch_size = 10000
num_epochs=100
learning_rate=1e-3
# weight of KL loss term
beta = 0.1
train(beer_ae, recipes_scaled, batch_size=batch_size, num_epochs=num_epochs, learning_rate=learning_rate, beta=beta)
#train(beer_ae, recipes.values, batch_size=batch_size, num_epochs=num_epochs, learning_rate=learning_rate, beta=beta)

/home/jupyter-rory/repos/beer.ai/env/lib/python3.8/site-packages/torch/nn/functional.py:2398: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


epoch [1/100], loss:0.249381
epoch [2/100], loss:0.247609
epoch [3/100], loss:0.245421
epoch [4/100], loss:0.242263
epoch [5/100], loss:0.237759
epoch [6/100], loss:0.231570
epoch [7/100], loss:0.223319
epoch [8/100], loss:0.212647
epoch [9/100], loss:0.199245
epoch [10/100], loss:0.182827
epoch [11/100], loss:0.163280
epoch [12/100], loss:0.140818
epoch [13/100], loss:0.116047
epoch [14/100], loss:0.090176
epoch [15/100], loss:0.065023
epoch [16/100], loss:0.042750
epoch [17/100], loss:0.025224
epoch [18/100], loss:0.013237
epoch [19/100], loss:0.006186
epoch [20/100], loss:0.002592
epoch [21/100], loss:0.000970
epoch [22/100], loss:0.000305
epoch [23/100], loss:0.000052
epoch [24/100], loss:-0.000042
epoch [25/100], loss:-0.000079
epoch [26/100], loss:-0.000097
epoch [27/100], loss:-0.000104
epoch [28/100], loss:-0.000104
epoch [29/100], loss:-0.000100
epoch [30/100], loss:-0.000099
epoch [31/100], loss:-0.000101
epoch [32/100], loss:-0.000105
epoch [33/100], loss:-0.000106
epoch [34

KeyboardInterrupt: 

In [12]:
decoded = beer_ae.decode(beer_ae.encode(recipes.values).values)
decoded.index = recipes.index

In [13]:
decoded[decoded < 1e-6] = 0

In [14]:
decoded

,0,1,2,3,4,5,6,7,8,9,...,678,679,680,681,682,683,684,685,686,687
recipe_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604217
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604218
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.605101
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604216
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604216
404604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604217
404606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604217


In [15]:
np.sqrt((recipes.values - decoded.values)**2).mean()

0.11336676649631938

In [16]:
random_beer = beer_ae.decode(np.random.rand(50))
random_beer = scaler.inverse_transform(random_beer.T)[0]

In [17]:
random_beer[random_beer < 0.0001] = 0

In [18]:
vocab, inv_vocab = load_vocab()

In [23]:
ing_ids = np.where(random_beer > 0)[0]

In [24]:
ing_ids

array([  0,   1,   2,   3,   4,   5,   6,   7,   9,  10,  11,  12,  13,
        14,  15,  16,  18,  19,  20,  21,  22,  23,  24,  25,  26,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  38,  39,  40,  41,  42,
        43,  44,  45,  48,  49,  51,  52,  53,  54,  55,  56,  57,  59,
        60,  61,  62,  63,  64,  66,  67,  68,  69,  71,  72,  73,  74,
        76,  77,  78,  79,  81,  82,  83,  84,  85,  86,  88,  89,  92,
        93,  94,  95, 100, 103, 104, 106, 107, 109, 110, 111, 112, 113,
       116, 117, 118, 120, 125, 129, 130, 131, 135, 141, 142, 143, 145,
       149, 150, 152, 153, 154, 155, 156, 157, 159, 161, 164, 166, 167,
       178, 180, 181, 182, 185, 186, 187, 188, 193, 196, 199, 201, 205,
       209, 210, 211, 212, 213, 218, 219, 220, 224, 225, 228, 229, 230,
       231, 235, 236, 237, 240, 242, 248, 252, 253, 254, 256, 258, 260,
       261, 267, 269, 271, 273, 275, 277, 278, 282, 284, 285, 287, 296,
       297, 308, 317, 320, 322, 329, 331, 334, 343, 345, 366, 38

In [25]:
for ing in ing_ids:
    print(inv_vocab[ing])

ferm_2-row
ferm_6-row
ferm_abbey malt
ferm_acidulated malt
ferm_amber malt
ferm_aromatic barley malt
ferm_aromatic® (munich) malt
ferm_biscuit® md™
ferm_black malt
ferm_blackprinz
ferm_brown malt
ferm_brown sugar
ferm_brown sugar, dark
ferm_brown sugar, light
ferm_candi sugar, amber
ferm_candi sugar, clear
ferm_cane sugar
ferm_cara malt
ferm_caraamber
ferm_caraaroma
ferm_carabelge
ferm_caracrystal wheat malt
ferm_carafa i
ferm_carafa ii
ferm_carafa iii
ferm_carafa special iii
ferm_carafoam
ferm_carahell®
ferm_caramel malt 10l
ferm_caramel malt 120l
ferm_caramel malt 20l
ferm_caramel malt 40l
ferm_caramel malt 80l
ferm_caramel/crystal 150l
ferm_caramel/crystal 60l
ferm_caramel/crystal 90l
ferm_caramunich
ferm_caramunich i
ferm_caramunich ii
ferm_caramunich iii
ferm_carapils® malt
ferm_carared
ferm_chocolate
ferm_chocolate malt
ferm_chocolate wheat
ferm_coffee malt
ferm_corn sugar (dextrose)
ferm_crystal malt
ferm_crystal rye
ferm_dark chocolate
ferm_dark dry malt extract
ferm_dry malt e

KeyError: 687

# Ideas from Ethan


## Model Ideas

* Look at adding regularization in between layers
* Look at VAE
* Look at GAN
  * This is most interesting
  * Its explicit job is creating recipes that look real.
  * Discriminator needs to have some function that estimates "quality" of what is produced (decoded space).

Combining the properties of a VAE (where you can interpolate) and a GAN (where you can estimate "quality" of a recipe) would be perfect.

Main issue is that reconstruction error doesn't give you the properties you want. Look for other learning signals - what other aspect of a recipe is meaningful? Could we use the beer style as a label?


## Encoding Ideas

* Can you encode hops into all possible combinations present in the dataset (not all combinations period)? Rob thinks this is reasonable
* Can we make ingredient "categories"? e.g. caramel with varying lovabond measurement?